# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`MET-Data`*

---
*@author: Evan*\
*@date: 2023-07-07*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import sys
sys.path.append('../../src/')
from namelist import *

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
pa = xr.open_dataset('D:/data/Project_Anqing/202306/COMBINE_ACONC_v54_ISAM_intel_CN3AH_135X138_2023_pa.nc')
grid = xr.open_dataset('D:/data/Project_Anqing/GRIDCRO2D_2023141.nc')

In [3]:
# 将时间点转换为实际时间
times=pd.date_range(timestart,timeend,freq='h')
times

DatetimeIndex(['2023-06-01 00:00:00', '2023-06-01 01:00:00',
               '2023-06-01 02:00:00', '2023-06-01 03:00:00',
               '2023-06-01 04:00:00', '2023-06-01 05:00:00',
               '2023-06-01 06:00:00', '2023-06-01 07:00:00',
               '2023-06-01 08:00:00', '2023-06-01 09:00:00',
               ...
               '2023-06-30 14:00:00', '2023-06-30 15:00:00',
               '2023-06-30 16:00:00', '2023-06-30 17:00:00',
               '2023-06-30 18:00:00', '2023-06-30 19:00:00',
               '2023-06-30 20:00:00', '2023-06-30 21:00:00',
               '2023-06-30 22:00:00', '2023-06-30 23:00:00'],
              dtype='datetime64[ns]', length=720, freq='H')

In [4]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [5]:
days=1 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from PA
        HADV_O3=(['time','level','y','x'],pa.HADV_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Horizontal Advection of O3','units':'ug m-3'}),
        ZADV_O3=(['time','level','y','x'],pa.ZADV_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Veritical Advection of O3','units':'ug m-3'}),
        HDIF_O3=(['time','level','y','x'],pa.HDIF_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Horizontal Diffusion of O3','units':'ug m-3'}),
        VDIF_O3=(['time','level','y','x'],pa.VDIF_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Vertical Diffusion of O3','units':'ug m-3'}),
        DDEP_O3=(['time','level','y','x'],pa.DDEP_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Dry Deposition of O3','units':'ug m-3'}),
        CHEM_O3=(['time','level','y','x'],pa.CHEM_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Gas-phase Chemistry of O3','units':'ug m-3'}),

        HNO3prod=(['time','level','y','x'],pa.HNO3prod[days*24-8:-8,:,:,:].data,{'long name':'Production Velocity of HNO3','units':'ppbv'}),
        H2O2prod=(['time','level','y','x'],pa.H2O2prod[days*24-8:-8,:,:,:].data,{'long name':'Production Velocity of H2O2','units':'ppbv'}),
        O3prod=(['time','level','y','x'],pa.O3prod[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Production Velocity of O3','units':'ug m-3'}),
        O3loss=(['time','level','y','x'],pa.O3loss[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Loss Velocity of O3','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202306',
        grid='CN3AH_135X138',
        createtime=pd.Timestamp.now().strftime('%Y-%m-%d'),
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 720, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-06-01 ... 2023-06-30T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    HADV_O3    (time, level, y, x) float32 -83.73 3.384 -260.8 ... -51.89 -43.89
    ZADV_O3    (time, level, y, x) float32 80.67 10.23 272.6 ... -5.242 -12.84
    HDIF_O3    (time, level, y, x) float32 0.3103 -0.3068 ... 0.9352 0.2411
    VDIF_O3    (time, level, y, x) float32 3.59 5.025 87.21 ... -0.03237 -0.0332
    DDEP_O3    (time, level, y, x) float32 -2.3 -22.09 -96.81 ... 0.0 0.0 0.0
    CHEM_O3    (time, level, y, x) float32 -3.14 -2.03 -1.431 ... 0.0 0.0 0.0
    HNO3prod   (time, level, y, x) float32 0.1172 0.1171 ... 2.888e-05 2.899e-05
    H2O2prod   (time, level, y, x) float32 0.0036 0.001513 ... 1.12e-08 1.12e-08
    O3prod     (time, level, y, x) float32 0.0004818 0.0002669 ... 1.908e-11
    O3loss     (time, level, y, x) float32 3.141 2.03 ... 0.001173 0.001176
Attributes:
    case:        Anqing_202306
    grid:        CN3AH_135X138
    createtime:  2023-07-07

In [6]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/AQ_2306_pa.nc',encoding=encoding)